In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.getcwd() # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

# Drop unnecessary columns
if "TXT_file_name" in df.columns:
    df = df.drop(columns=["TXT_file_name"])

# Handle missing values
df = df.dropna(subset=["question"])
df["context"].fillna("", inplace=True)

# Clean text
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\b[a-z]\)\s+', ' ', text)  # Remove patterns like 'a)', 'b)', etc.
    text = re.sub(r'\b\d+\.\b', '', text)  # Remove patterns like '1.', '2.', etc.
    text = re.sub(r'\b\d+\)\b', '', text)  # Remove patterns like '1)', '2)', etc.
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)  # Remove patterns like 'i.', 'ii.', 'i)', etc.
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    text = re.sub(r'\b\d+[.)]\s*', '', text) # Remove numeric list markers like 1., 2. or 1) 2)
    text = re.sub(r'\b[ivxlcdm]+\s*[.)]\s*', '', text, flags=re.IGNORECASE)# Remove roman numerals like i. ii. iii. or i) ii) iii)
    return text

# df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)
df["clean_text"] = (df["question"]).apply(clean_text) 

# Group by 'clean_text' and count unique themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]

# Merge back with the original dataframe to get all rows with these 'clean_text'
filtered_df = df[df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]
# Exclude rows with these 'clean_text' from the original dataframe
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]

# ✅ Now: drop rare themes using original theme names
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme"].isin(valid_themes)]

# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

print("All theme_ids:", sorted(df["theme_id"].unique()))
print("num_labels:", df["theme_id"].nunique())


C:\Users\corne\AppData\Local\Temp\ipykernel_29564\1437698013.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)


All theme_ids: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36)]
num_labels: 37


In [4]:
# ✅ 5. Split Data into Train & Test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["clean_text"].tolist(), df["theme_id"].tolist(), test_size=0.2, random_state=42, stratify=df["theme_id"]
)

from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

# Compute class counts and use median as balancing target
theme_counts = train_df["theme_id"].value_counts()
median_count = theme_counts.median()

# Define strategy: only oversample underrepresented classes
sampling_strategy = {
    theme: int(median_count)
    for theme in theme_counts.index
    if theme_counts[theme] < median_count
}

# Apply RandomOverSampler
ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = ros.fit_resample(train_df[["clean_text"]], train_df["theme_id"])

# Extract oversampled train lists
train_texts_resampled = X_resampled["clean_text"].tolist()
train_labels_resampled = y_resampled.tolist()

from collections import Counter
print("Class distribution after oversampling:", Counter(train_labels_resampled))


# ✅ 7. Load BERT Tokenizer & Define Dataset Class
model_name = "GroNLP/bert-base-dutch-cased"              # aanpassen naar welk model je wilt evaluaten 
tokenizer = BertTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts_resampled, train_labels_resampled, tokenizer)
test_dataset = ThemeDataset(test_texts, test_labels, tokenizer)

Class distribution after oversampling: Counter({15: 4841, 4: 1521, 9: 1455, 26: 1066, 12: 972, 6: 776, 21: 742, 20: 635, 0: 602, 16: 573, 28: 560, 14: 539, 8: 449, 1: 398, 17: 343, 11: 314, 22: 281, 23: 261, 7: 233, 35: 233, 18: 233, 19: 233, 32: 233, 29: 233, 24: 233, 5: 233, 31: 233, 13: 233, 2: 233, 30: 233, 25: 233, 3: 233, 27: 233, 10: 233, 36: 233, 33: 233, 34: 233})


LOAD THE MODEL YOU WANT TO EVALUATE FROM KUL DRIVE!!

In [5]:
print("Train dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))

Train dataset size: 20755
Test dataset size: 4583


In [6]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = "C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP/Run_2025-04-07_23-42"        # ook naam tokenize in cel hierboven aanpassen !!!
#model_path = r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\BERT_Classifiers\results\checkpoint-13790"


model = BertForSequenceClassification.from_pretrained(model_path).cuda()
print(model.config._name_or_path)


C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP/Run_2025-04-07_23-42


In [7]:
training_args = TrainingArguments(output_dir="./results", per_device_eval_batch_size=8)

trainer = Trainer(
    model=model,
    args=training_args
)

In [8]:
predictions = trainer.predict(test_dataset)


100%|██████████| 573/573 [01:18<00:00,  7.33it/s]


HANDLE UNKNOWNS and save those predictions

In [ ]:
import torch
import torch.nn.functional as F
import pandas as pd

# Step 2: Convert logits to softmax probabilities
logits = torch.tensor(predictions.predictions)
probs = F.softmax(logits, dim=1)

# Step 3: Extract top confidence and predicted label
confidences, pred_ids = torch.max(probs, dim=1)

# Step 4: Create DataFrame for analysis
df_conf = pd.DataFrame({
    "question": test_texts,  # should match the input order of test_dataset
    "true_label_id": predictions.label_ids,
    "predicted_label_id": pred_ids.numpy(),
    "confidence": confidences.numpy()
})

# Step 5: Add theme names (optional)
df_conf["true_theme"] = df_conf["true_label_id"].map(id_to_theme)
df_conf["predicted_theme"] = df_conf["predicted_label_id"].map(id_to_theme)

# Step 6: Sort by confidence to see vague cases
df_conf_sorted = df_conf.sort_values("confidence")

# Save or inspect
df_conf_sorted.to_excel("bert_predictions_with_confidence.xlsx", index=False)
df_conf_sorted.head(10)  # Show least confident predictions





✅ Saved to low_confidence_predictions.csv


In [31]:
# ✅ 13. Make Predictions (With Dynamic Confidence Threshold & Short Question Handling)
probabilities = F.softmax(torch.tensor(predictions.predictions), dim=1)

# ✅ Dynamically Adjust the Confidence Threshold (1st Percentile)
confidence_values = torch.max(probabilities, dim=1)[0].tolist()
dynamic_threshold = np.percentile(confidence_values, 1)  # ✅ Set threshold at the 5th percentile
print(f"Dynamic Threshold: {dynamic_threshold}")  # ✅ Print the new threshold

# ✅ Predict Themes with "Unknown" for Unclear Questions
predicted_labels = []
for i in range(len(probabilities)):
    max_prob = torch.max(probabilities[i]).item()
    pred_label = torch.argmax(probabilities[i]).item()
    question_text = test_texts[i]

    # ✅ If question is too short and lacks context, assign "Unknown"
    if len(question_text.split()) < 4:
        predicted_labels.append("Unknown")
    elif max_prob < dynamic_threshold:
        predicted_labels.append("Unknown")  # ✅ Filter out low-confidence predictions
    else:
        predicted_labels.append(id_to_theme[pred_label])  # ✅ Assign label


Dynamic Threshold: 0.41512388348579404


In [32]:
# Create list of results
results = [
    "Unknown" if pred == "Unknown"
    else "Correct" if id_to_theme[true_label] == pred
    else "Incorrect"
    for true_label, pred in zip(test_labels, predicted_labels)
]

# Create DataFrame with results column
output_df = pd.DataFrame({
    "Text": test_texts,
    "True_Theme": [id_to_theme[label] for label in test_labels],
    "Predicted_Theme": predicted_labels,
    "Result": results
})

# Save to Excel
output_df.to_excel("test.xlsx", index=False)
print("✅ Model Training Completed! Predictions saved.")


✅ Model Training Completed! Predictions saved.
